# 과거 미국 주식 랠리 분석 — 다음 랠리의 승자는 누구인가?

## 배경

2026년 하반기, 중간선거를 앞두고 미국 주식시장의 랠리가 예상된다. 이후 AI 버블이 본격화될 가능성까지 고려하면, 지금 시점에서 **"랠리 때 어떤 종목이 주로 상승하는가"** 를 과거 데이터로 검증해볼 필요가 있다.

분석 대상으로 **1988~2000년 닷컴 버블 랠리**를 선택했다. 기술 혁신이 주도한 장기 상승장이라는 점에서 현재 AI 랠리와 구조적 유사성이 있기 때문이다.

## 질문 설정

1. 12년간의 랠리에서 실제로 큰 수익을 낸 종목은 어떤 특징을 갖고 있는가?
2. 테크 섹터만 올랐는가, 아니면 다른 섹터에서도 승자가 나왔는가?
3. 랠리 말기(1999~2000)에 새로 진입한 종목과 장기 생존 종목의 차이는?
4. 이 패턴이 현재 AI 랠리에 시사하는 점은?

## 분석 대상

| 구분 | 기간 | 설명 |
|------|------|------|
| 그룹 1 (Full Coverage) | 1988.10 ~ 2000.03 | 12년 전체 랠리를 경험한 장기 생존 종목 162개 |
| 그룹 2 (Late Entry) | 1999.03 ~ 2000.03 | 버블 말기 1년간 데이터가 있는 종목 (신규 상장 포함) |

## 한계

- 현재 상장 종목 기준으로 수집했기 때문에 **Survivorship Bias**가 존재한다 (1988~2000 사이 상장폐지된 종목 누락)
- YHOO(Yahoo), SUNW(Sun Microsystems) 등 닷컴 버블 핵심 종목이 분석에서 빠져 있다
- 추후 FMP API로 데이터 소스를 교체하여 상장폐지 종목까지 포함할 예정

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings('ignore')

# 분석 기간 설정
START_DATE = "1988-10-01"
END_DATE = "2000-03-31"

print(f"분석 기간: {START_DATE} ~ {END_DATE}")

## 1. 데이터 수집

NASDAQ Trader에서 현재 상장된 전체 종목(NASDAQ + NYSE 등) 티커를 수집하고, 주요 지수 심볼을 추가한다.

In [ ]:
all_tickers = set()

# --- NASDAQ Trader FTP - 전체 상장 종목 ---
try:
    # NASDAQ 거래소 상장 종목
    nasdaq_listed = pd.read_csv(
        "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt",
        sep="|"
    )
    nasdaq_listed = nasdaq_listed[nasdaq_listed["Test Issue"] == "N"]
    nasdaq_syms = set(nasdaq_listed["Symbol"].dropna().tolist())
    # 기타 거래소 (NYSE 등)
    other_listed = pd.read_csv(
        "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt",
        sep="|"
    )
    if "Test Issue" in other_listed.columns:
        other_listed = other_listed[other_listed["Test Issue"] == "N"]
    if "ACT Symbol" in other_listed.columns:
        other_syms = set(other_listed["ACT Symbol"].dropna().tolist())
    elif "Symbol" in other_listed.columns:
        other_syms = set(other_listed["Symbol"].dropna().tolist())
    else:
        other_syms = set()
    
    exchange_tickers = nasdaq_syms | other_syms
    # 일반 주식만 필터링 (ETF, 워런트 등 제외를 위해 특수문자 포함 티커 제거)
    exchange_tickers = {t for t in exchange_tickers if isinstance(t, str) and t.isalpha() and len(t) <= 5}
    all_tickers.update(exchange_tickers)
    print(f"NASDAQ Trader (NASDAQ + 기타 거래소): {len(exchange_tickers)}개")
except Exception as e:
    print(f"NASDAQ Trader 수집 실패: {e}")

# --- 주요 지수 심볼 추가 ---
index_tickers = {"^GSPC", "^DJI", "^IXIC", "^RUT", "^NYA"}
all_tickers.update(index_tickers)

print(f"\n총 고유 티커 수: {len(all_tickers)}개")

In [ ]:
# 티커 길이 분포 확인
from collections import Counter

lengths = Counter(len(t) for t in all_tickers if not t.startswith("^"))
print("티커 길이 분포:")
for l in sorted(lengths):
    print(f"  {l}글자: {lengths[l]}개")

# 알파벳만으로 구성된 티커 vs 숫자/특수문자 포함 티커
alpha_only = [t for t in all_tickers if t.isalpha()]
non_alpha = [t for t in all_tickers if not t.isalpha() and not t.startswith("^")]
print(f"\n알파벳만: {len(alpha_only)}개")
print(f"숫자/특수문자 포함: {len(non_alpha)}개")
print(f"지수(^): 5개")
print(f"\n숫자/특수문자 포함 샘플:")
print(sorted(non_alpha)[:20])

In [ ]:
print(f"총 티커 수: {len(all_tickers)}")
print(f"\n샘플 티커 (정렬 후 처음 30개):")
print(sorted(all_tickers)[:30])
print(f"\n샘플 티커 (정렬 후 마지막 30개):")
print(sorted(all_tickers)[-30:])

In [ ]:
# 잘 알려진 대형주가 포함되어 있는지 확인
major_stocks = ["AAPL", "MSFT", "INTC", "CSCO", "GE", "IBM", "JPM", "XOM", 
                "WMT", "KO", "PG", "JNJ", "MRK", "PFE", "BA", "DIS",
                "AMZN", "ORCL", "DELL", "YHOO", "SUNW", "CMGI"]

found = [t for t in major_stocks if t in all_tickers]
missing = [t for t in major_stocks if t not in all_tickers]

print(f"주요 대형주 포함 여부:")
print(f"  포함: {found}")
print(f"  누락: {missing}")

# 1988-2000 랠리 시기 핵심 종목 중 상장폐지된 종목 확인
# (DELL, YHOO, SUNW, CMGI 등은 상장폐지/인수되어 현재 목록에 없을 수 있음)
print(f"\n※ 누락된 종목은 상장폐지/인수/티커 변경된 종목일 가능성 높음")

In [ ]:
# ETF가 섞여있는지 확인 (1988-2000 분석에 ETF는 불필요할 수 있음)
known_etfs = ["SPY", "QQQ", "IWM", "DIA", "XLK", "XLF", "XLE", "XLV", 
              "VTI", "VOO", "IVV", "AGG", "BND", "GLD", "SLV", "ARKK"]
etfs_found = [t for t in known_etfs if t in all_tickers]
print(f"포함된 알려진 ETF: {etfs_found}")
print("(대부분 1998년 이후 상장이라 해당 기간 데이터가 제한적일 수 있음)")

# 1글자 티커 확인 (보통 대형 우량주)
one_char = sorted([t for t in all_tickers if len(t) == 1])
print(f"\n1글자 티커 ({len(one_char)}개): {one_char}")

## 2. yfinance 배치 다운로드

11,724개 티커를 100개씩 배치로 나눠 1988.10~2000.03 기간의 종가 데이터를 다운로드한다. 현재 상장 종목 기준이므로 해당 기간에 존재하지 않았던 종목은 데이터가 없다.

In [ ]:
def download_batch(tickers, start, end, batch_size=100):
    """티커를 배치로 나눠서 yfinance에서 다운로드"""
    ticker_list = sorted(list(tickers))
    all_data = {}
    failed = []
    
    total_batches = (len(ticker_list) + batch_size - 1) // batch_size
    
    for i in range(0, len(ticker_list), batch_size):
        batch = ticker_list[i:i+batch_size]
        batch_num = i // batch_size + 1
        print(f"배치 {batch_num}/{total_batches} 다운로드 중... ({len(batch)}개 티커)")
        
        try:
            data = yf.download(
                tickers=batch,
                start=start,
                end=end,
                auto_adjust=True,
                progress=False,
                threads=True
            )
            
            if data.empty:
                continue
            
            # 단일 티커인 경우 MultiIndex가 아님
            if len(batch) == 1:
                ticker = batch[0]
                if not data.empty:
                    all_data[ticker] = data["Close"]
            else:
                # MultiIndex columns: (Price, Ticker)
                if isinstance(data.columns, pd.MultiIndex):
                    close_data = data["Close"]
                else:
                    close_data = data
                
                for ticker in close_data.columns:
                    col = close_data[ticker].dropna()
                    if len(col) > 0:
                        all_data[ticker] = col
                        
        except Exception as e:
            failed.extend(batch)
            print(f"  배치 {batch_num} 실패: {e}")
        
        # API rate limit 방지
        if batch_num % 5 == 0:
            time.sleep(1)
    
    print(f"\n다운로드 완료: {len(all_data)}개 종목 데이터 확보")
    if failed:
        print(f"실패한 티커: {len(failed)}개")
    
    return all_data

# 다운로드 실행
stock_data = download_batch(all_tickers, START_DATE, END_DATE, batch_size=100)

## 3. 데이터 필터링

11,724개 중 360개만 해당 기간 데이터가 존재했다. 대부분의 현재 상장 종목은 2000년 이후 IPO이므로 정상적인 결과다. 최소 250거래일(약 1년) 이상 데이터를 보유한 339개 종목을 분석 대상으로 선정한다.

In [ ]:
# 최소 거래일 수 기준으로 필터링 (최소 250일 = 약 1년)
MIN_TRADING_DAYS = 250

valid_tickers = {k: v for k, v in stock_data.items() if len(v) >= MIN_TRADING_DAYS}

print(f"전체 다운로드 종목: {len(stock_data)}개")
print(f"최소 {MIN_TRADING_DAYS}거래일 이상 데이터 보유: {len(valid_tickers)}개")

# 종합 DataFrame 생성
close_df = pd.DataFrame(valid_tickers)
close_df.index = pd.to_datetime(close_df.index)
close_df = close_df.sort_index()

print(f"\n데이터 기간: {close_df.index[0].date()} ~ {close_df.index[-1].date()}")
print(f"총 거래일: {len(close_df)}")
print(f"종목 수: {close_df.shape[1]}")

# 기간별 커버리지 확인
coverage = close_df.notna().sum(axis=1)
print(f"\n일자별 평균 종목 수: {coverage.mean():.0f}")
print(f"일자별 최소 종목 수: {coverage.min()} ({coverage.idxmin().date()})")
print(f"일자별 최대 종목 수: {coverage.max()} ({coverage.idxmax().date()})")

# 수익률 계산
returns_df = close_df.pct_change()

# 기간 전체 수익률 (처음~끝)
total_returns = {}
for col in close_df.columns:
    series = close_df[col].dropna()
    if len(series) >= 2:
        total_returns[col] = (series.iloc[-1] / series.iloc[0] - 1) * 100

total_returns_series = pd.Series(total_returns).sort_values(ascending=False)

print(f"\n=== 기간 전체 수익률 상위 20 ===")
print(total_returns_series.head(20).to_string(float_format="{:.1f}%".format))
print(f"\n=== 기간 전체 수익률 하위 10 ===")
print(total_returns_series.tail(10).to_string(float_format="{:.1f}%".format))

In [ ]:
close_df.head()

In [ ]:
close_df.describe()

In [ ]:
# 종목별 결측치 현황
# 분석 기간: 1988-10-01 ~ 2000-03-31
missing_per_stock = close_df.isna().sum().sort_values(ascending=False)
total_days = len(close_df)

print(f"총 거래일: {total_days}")
print(f"\n=== 종목별 결측치 상위 20 ===")
for ticker, missing in missing_per_stock.head(20).items():
    pct = missing / total_days * 100
    first_valid = close_df[ticker].first_valid_index()
    print(f"  {ticker:6s}: {missing:4d}일 결측 ({pct:5.1f}%)  | 첫 데이터: {first_valid.date()}")

print(f"\n=== 결측 없는 종목 수: {(missing_per_stock == 0).sum()}개")
print(f"=== 결측 1~100일: {((missing_per_stock > 0) & (missing_per_stock <= 100)).sum()}개")
print(f"=== 결측 100~500일: {((missing_per_stock > 100) & (missing_per_stock <= 500)).sum()}개")
print(f"=== 결측 500일 이상: {(missing_per_stock > 500).sum()}개")


In [ ]:
missing_per_stock.head()

In [ ]:
# 결측치 처리: 기간별 분리
# 첫 번째 분석 대상: 1988-10-01 ~ 2000-03-31 기간에 결측치가 없는 데이터: 162개 종목
close_df_1 = close_df.loc["1988-10-01":"2000-03-31"]
close_df_1 = close_df_1.dropna(axis=1)
print(f"첫 번째 분석 대상 종목 수: {close_df_1.shape[1]}개")

# 두 번째 분석 대상: 1999-03-31 ~ 2000-12-31 기간에 결측치가 없는 데이터: 339개 종목
close_df_2 = close_df.loc["1999-03-31":"2000-12-31"]
close_df_2 = close_df_2.dropna(axis=1) # 만약을 대비
print(f"두 번째 분석 대상 종목 수: {close_df_2.shape[1]}개")

In [ ]:
close_df_1.describe()

In [ ]:
close_df_2.describe()

In [ ]:
close_df_1.head()

## 4. 시각화 — 162개 풀 커버리지 종목의 랠리 흐름

339개 종목 중 12년 전체 기간에 결측치가 없는 162개 종목(그룹 1)을 시작가=100으로 정규화하여 시각화한다. 개별 종목은 배경으로, 전체 중앙값을 기준선으로 표시한다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# example
x = close_df_1.index
y = close_df_1['AAPL']

plt.plot(x, y)

In [ ]:
# 정규화 후 중앙값
normalized_1 = close_df_1 / close_df_1.iloc[0] * 100
plt.plot(normalized_1, color='grey', alpha=0.4)
plt.plot(normalized_1.median(axis=1), color='red', linewidth=2, label='Median')
plt.legend()


In [ ]:
# 두 번째 그룹
normalized_2 = close_df_2 / close_df_2.iloc[0] * 100
plt.plot(normalized_2, color='grey', alpha=0.4)
plt.plot(normalized_2.median(axis=1), color='red', linewidth=2, label='Median')
plt.legend()

In [ ]:
normalized_1 = close_df_1 / close_df_1.iloc[0] * 100
med = normalized_1.median(axis=1)
avg = normalized_1.mean(axis=1)

print(f"중앙값 - 시작: {med.iloc[0]:.1f}, 끝: {med.iloc[-1]:.1f}, 배수: {med.iloc[-1]/med.iloc[0]:.2f}x")
print(f"평균값 - 시작: {avg.iloc[0]:.1f}, 끝: {avg.iloc[-1]:.1f}, 배수: {avg.iloc[-1]/avg.iloc[0]:.2f}x")

# 개별 종목 최종 정규화 값 분포
final_vals = normalized_1.iloc[-1].dropna().sort_values()
print(f"\n=== 162개 종목 최종 정규화 값 분포 ===")
print(f"최소: {final_vals.iloc[0]:.1f} ({final_vals.index[0]})")
print(f"25%: {final_vals.quantile(0.25):.1f}")
print(f"중앙: {final_vals.median():.1f}")
print(f"75%: {final_vals.quantile(0.75):.1f}")
print(f"최대: {final_vals.iloc[-1]:.1f} ({final_vals.index[-1]})")
print(f"평균: {final_vals.mean():.1f}")

In [ ]:
normalized_2 = close_df_2 / close_df_2.iloc[0] * 100
med = normalized_2.median(axis=1)
avg = normalized_2.mean(axis=1)

print(f"중앙값 - 시작: {med.iloc[0]:.1f}, 끝: {med.iloc[-1]:.1f}, 배수: {med.iloc[-1]/med.iloc[0]:.2f}x")
print(f"평균값 - 시작: {avg.iloc[0]:.1f}, 끝: {avg.iloc[-1]:.1f}, 배수: {avg.iloc[-1]/avg.iloc[0]:.2f}x")

# 개별 종목 최종 정규화 값 분포
final_vals = normalized_2.iloc[-1].dropna().sort_values()
print(f"\n=== 162개 종목 최종 정규화 값 분포 ===")
print(f"최소: {final_vals.iloc[0]:.1f} ({final_vals.index[0]})")
print(f"25%: {final_vals.quantile(0.25):.1f}")
print(f"중앙: {final_vals.median():.1f}")
print(f"75%: {final_vals.quantile(0.75):.1f}")
print(f"최대: {final_vals.iloc[-1]:.1f} ({final_vals.index[-1]})")
print(f"평균: {final_vals.mean():.1f}")

### 로그 스케일 + 섹터별 색상

가격 편차가 100배 이상이므로 로그 스케일을 적용해야 수익률 흐름이 정확히 보인다. 개별 종목에 섹터별 색상을 입혀 **질문 2: "테크 섹터만 올랐는가?"** 를 시각적으로 확인한다.

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
plt.plot(normalized_1, color='grey', alpha=0.4)
plt.plot(normalized_1.median(axis=1), color='red', linewidth=2, label='Median')
plt.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
plt.plot(normalized_2, color='grey', alpha=0.4)
plt.plot(normalized_2.median(axis=1), color='red', linewidth=2, label='Median')
plt.legend()

In [ ]:
# 섹터 정보 수집
sectors = {}
for ticker in close_df_1.columns:
    try:
        info = yf.Ticker(ticker).info
        sectors[ticker] = info.get('sector', 'Unknown')
    except Exception as e:
        sectors[ticker] = 'Unknown'
sector_df = pd.Series(sectors)
sector_df.head(10)

In [ ]:
# 섹터 정보 수집: 두 번째 그룹
sectors_2 = {}
for ticker in close_df_2.columns:
    try:
        info = yf.Ticker(ticker).info
        sectors_2[ticker] = info.get('sector', 'Unknown')
    except Exception as e:
        sectors_2[ticker] = 'Unknown'
sector_df_2 = pd.Series(sectors_2)
sector_df_2.head(10)

In [ ]:
# 첫 번째 그룹
sector_df.unique()

In [ ]:
# 두 번째 그룹
sector_df_2.unique()

In [ ]:
# 섹터별 색상 그래프: 첫 번째 그룹
fig, ax = plt.subplots(figsize=(14, 7))
ax.set_yscale('log')

# 섹터별 색상 매핑
unique_sectors = sector_df.unique()
cmap = plt.cm.get_cmap('tab10', len(unique_sectors))
sector_colors = {sector: cmap(i) for i, sector in enumerate(sorted(unique_sectors))}

# 섹터별로 개별 종목 그리기
for sector, tickers in sector_df.groupby(sector_df):
    color = sector_colors[sector]
    for ticker in tickers.index:
        if ticker in normalized_1.columns:
            ax.plot(normalized_1[ticker], color=color, alpha=0.3, linewidth=0.7)
    # 범례용 더미 라인 (섹터당 1개)
    ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({len(tickers)})')

# 중앙값 선
ax.plot(normalized_1.median(axis=1), color='black', linewidth=2.5, linestyle='--', label='Median')

ax.legend(fontsize=8, loc='upper left', ncol=2)
ax.set_ylabel('Normalized Price (Start=100, Log Scale)')
ax.set_title('1988.10 ~ 2000.03 Rally by Sector (162 stocks)')
plt.tight_layout()
plt.show()

In [ ]:
# 섹터별 색상 그래프: 두 번째 그룹
fig, ax = plt.subplots(figsize=(14, 7))
ax.set_yscale('log')

# 섹터별 색상 매핑
unique_sectors = sector_df_2.unique()
cmap = plt.cm.get_cmap('tab10', len(unique_sectors))
sector_colors = {sector: cmap(i) for i, sector in enumerate(sorted(unique_sectors))}

# 섹터별로 개별 종목 그리기
for sector, tickers in sector_df_2.groupby(sector_df_2):
    color = sector_colors[sector]
    for ticker in tickers.index:
        if ticker in normalized_2.columns:
            ax.plot(normalized_2[ticker], color=color, alpha=0.3, linewidth=0.7)
    # 범례용 더미 라인 (섹터당 1개)
    ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({len(tickers)})')

# 중앙값 선
ax.plot(normalized_2.median(axis=1), color='black', linewidth=2.5, linestyle='--', label='Median')

ax.legend(fontsize=8, loc='upper left', ncol=2)
ax.set_ylabel('Normalized Price (Start=100, Log Scale)')
ax.set_title('1988.10 ~ 2000.03 Rally by Sector (339 stocks)')
plt.tight_layout()
plt.show()

## 5. 클러스터링 — 질문 1: "큰 수익을 낸 종목의 특징은?"

로그 정규화된 가격 시계열을 K-Means로 클러스터링하여, 비슷한 시기에 비슷한 주가 흐름을 보인 종목들을 그룹화한다. 각 클러스터의 섹터 구성과 수익률 분포를 비교하여 **슈퍼 성장주의 공통점**을 찾는다.

### K-Means Clustering

Elbow Method와 Silhouette Score로 최적 클러스터 수를 결정한 뒤, 클러스터별 소속 종목을 섹터 색상으로 시각화한다.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
# 그룹 1
log_norm_1 = np.log(normalized_1)
X = StandardScaler().fit_transform(log_norm_1.T)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)
cluster_df = pd.DataFrame({'Ticker': normalized_1.columns, 'Cluster': clusters})
cluster_df.head(10)

In [ ]:
# 그룹 2
log_norm_2 = np.log(normalized_2)
X = StandardScaler().fit_transform(log_norm_2.T)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)
cluster_df_2 = pd.DataFrame({'Ticker': normalized_2.columns, 'Cluster': clusters})
cluster_df_2.head(10)

In [ ]:
# 그룹 1
# 적절한 클러스터 수 찾기 Elbow Method, Silhouette Score
from sklearn.metrics import silhouette_score

inertias = []
sil_scores = []
K_range = range(2, 11)

log_norm_1 = np.log(normalized_1)
X = StandardScaler().fit_transform(log_norm_1.T)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X, labels))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Elbow Method
axes[0].plot(K_range, inertias, marker='o')
axes[0].set_title('Elbow Method')
axes[0].set_xlabel('Number of clusters (k)')
axes[0].set_ylabel('Inertia')

# Silhouette Score
axes[1].plot(K_range, sil_scores, marker='o', color='orange')
axes[1].set_title('Silhouette Score')
axes[1].set_xlabel('Number of clusters (k)')
axes[1].set_ylabel('Score')

plt.tight_layout()
plt.show()

In [ ]:
# 그룹 2
# 적절한 클러스터 수 찾기 Elbow Method, Silhouette Score
from sklearn.metrics import silhouette_score

inertias = []
sil_scores = []
K_range = range(2, 11)

log_norm_2 = np.log(normalized_2)
X = StandardScaler().fit_transform(log_norm_2.T)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X, labels))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Elbow Method
axes[0].plot(K_range, inertias, marker='o')
axes[0].set_title('Elbow Method')
axes[0].set_xlabel('Number of clusters (k)')
axes[0].set_ylabel('Inertia')

# Silhouette Score
axes[1].plot(K_range, sil_scores, marker='o', color='orange')
axes[1].set_title('Silhouette Score')
axes[1].set_xlabel('Number of clusters (k)')
axes[1].set_ylabel('Score')

plt.tight_layout()
plt.show()

In [ ]:
# 그룹 1
# 최적 n_clusters = 4
log_norm_1 = np.log(normalized_1)
X = StandardScaler().fit_transform(log_norm_1.T)
kmeans = KMeans(n_clusters=4, random_state=42)

clusters = kmeans.fit_predict(X)
cluster_df = pd.DataFrame({'Ticker': normalized_1.columns, 'Cluster': clusters})
cluster_df.head(10)

In [ ]:
# 그룹 2
# 최적 n_clusters = 3
log_norm_2 = np.log(normalized_2)
X = StandardScaler().fit_transform(log_norm_2.T)
kmeans = KMeans(n_clusters=3, random_state=42)

clusters = kmeans.fit_predict(X)
cluster_df_2 = pd.DataFrame({'Ticker': normalized_2.columns, 'Cluster': clusters})
cluster_df_2.head(10)

In [ ]:
# 그룹 1 클러스터 시각화: 섹터별 색깔 매핑
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for c in range(kmeans.n_clusters):
    ax = axes[c]
    ax.set_yscale('log')
    
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    cluster_data = normalized_1[cluster_tickers]
    
    # 개별 종목을 섹터 색상으로 그리기
    for ticker in cluster_tickers:
        sector = sector_df.get(ticker, 'Unknown')
        color = sector_colors.get(sector, 'grey')
        ax.plot(cluster_data[ticker], color=color, alpha=0.3, linewidth=0.7)
    
    # 해당 클러스터의 중앙값 (군집 중앙점)
    cluster_median = cluster_data.median(axis=1)
    ax.plot(cluster_median, color='black', linewidth=2.5, linestyle='--', label=f'Cluster {c} Median')
    
    # 섹터 범례 (이 클러스터에 존재하는 섹터만)
    cluster_sectors = sector_df[cluster_tickers].value_counts()
    for sector, count in cluster_sectors.items():
        ax.plot([], [], color=sector_colors.get(sector, 'grey'), linewidth=2, label=f'{sector} ({count})')
    
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    ax.set_title(f'Cluster {c} ({len(cluster_tickers)} stocks)')
    ax.set_ylabel('Normalized Price (Log)')

plt.tight_layout()
plt.show()

In [ ]:
# 그룹 2 클러스터 시각화: 섹터별 색깔 매핑
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for c in range(kmeans.n_clusters):
    ax = axes[c]
    ax.set_yscale('log')
    
    cluster_tickers = cluster_df_2[cluster_df_2['Cluster'] == c]['Ticker']
    cluster_data = normalized_2[cluster_tickers]
    
    # 개별 종목을 섹터 색상으로 그리기
    for ticker in cluster_tickers:
        sector = sector_df_2.get(ticker, 'Unknown')
        color = sector_colors.get(sector, 'grey')
        ax.plot(cluster_data[ticker], color=color, alpha=0.3, linewidth=0.7)
    
    # 해당 클러스터의 중앙값 (군집 중앙점)
    cluster_median = cluster_data.median(axis=1)
    ax.plot(cluster_median, color='black', linewidth=2.5, linestyle='--', label=f'Cluster {c} Median')
    
    # 섹터 범례 (이 클러스터에 존재하는 섹터만)
    cluster_sectors = sector_df_2[cluster_tickers].value_counts()
    for sector, count in cluster_sectors.items():
        ax.plot([], [], color=sector_colors.get(sector, 'grey'), linewidth=2, label=f'{sector} ({count})')
    
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    ax.set_title(f'Cluster {c} ({len(cluster_tickers)} stocks)')
    ax.set_ylabel('Normalized Price (Log)')

plt.tight_layout()
plt.show()

In [ ]:
# 섹터 1 클러스터 요약
for c in range(kmeans.n_clusters):
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    cluster_data = normalized_1[cluster_tickers]
    
    # 최종 수익률
    final_vals = cluster_data.iloc[-1]
    
    print(f"=== Cluster {c} ({len(cluster_tickers)} stocks) ===")
    print(f"  중앙값 최종: {cluster_data.median(axis=1).iloc[-1]:.1f} (={cluster_data.median(axis=1).iloc[-1]/100:.1f}x)")
    print(f"  평균 최종:   {final_vals.mean():.1f} (={final_vals.mean()/100:.1f}x)")
    print(f"  범위:        {final_vals.min():.1f} ~ {final_vals.max():.1f}")
    
    # 섹터 구성
    cluster_sectors = sector_df[cluster_tickers].value_counts()
    print(f"  섹터 구성:")
    for sector, count in cluster_sectors.items():
        print(f"    {sector}: {count}개")
    
    # 대표 종목 (최종 정규화 값 기준 상위 3)
    top3 = final_vals.nlargest(3)
    print(f"  상위 3종목: {', '.join([f'{t}({v:.0f})' for t, v in top3.items()])}")
    print()


In [ ]:
# === 통계적 검증 ===
from scipy.stats import kruskal, chi2_contingency
import numpy as np

print("=" * 50)
print("통계적 검증: 클러스터 간 차이가 유의미한가?")
print("=" * 50)

# 1. Kruskal-Wallis 검정: 클러스터 간 수익률 차이
print("\n[1] Kruskal-Wallis 검정 (클러스터 간 수익률 차이)")
print("-" * 40)

cluster_returns = []
for c in range(kmeans.n_clusters):
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    final_vals = normalized_1[cluster_tickers].iloc[-1]
    cluster_returns.append(final_vals.values)

stat, p_value = kruskal(*cluster_returns)
print(f"H-statistic: {stat:.2f}")
print(f"p-value: {p_value:.2e}")
if p_value < 0.05:
    print("→ 결론: 클러스터 간 수익률 차이가 통계적으로 유의미함 (p < 0.05)")
else:
    print("→ 결론: 클러스터 간 수익률 차이가 유의미하지 않음")

# 2. 카이제곱 검정: 섹터-클러스터 연관성
print("\n[2] 카이제곱 검정 (섹터-클러스터 연관성)")
print("-" * 40)

# 섹터 × 클러스터 교차표 생성
cluster_sector_df = pd.DataFrame({
    'Cluster': cluster_df.set_index('Ticker')['Cluster'],
    'Sector': sector_df
}).dropna()

contingency_table = pd.crosstab(cluster_sector_df['Sector'], cluster_sector_df['Cluster'])
print("섹터 × 클러스터 교차표:")
print(contingency_table)
print()

chi2, p_value_chi, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square: {chi2:.2f}")
print(f"자유도: {dof}")
print(f"p-value: {p_value_chi:.2e}")
if p_value_chi < 0.05:
    print("→ 결론: 섹터와 클러스터 간 연관성이 통계적으로 유의미함 (p < 0.05)")
else:
    print("→ 결론: 섹터와 클러스터 간 연관성이 유의미하지 않음 (독립적)")


In [ ]:
# 3. Bootstrap 신뢰구간: 중앙값 수익률의 불확실성
print("\n[3] Bootstrap 신뢰구간 (중앙값 수익률)")
print("-" * 40)

from scipy.stats import bootstrap

# 전체 종목의 최종 수익률
all_final_returns = normalized_1.iloc[-1].values

# Bootstrap으로 중앙값의 95% 신뢰구간 계산
rng = np.random.default_rng(42)
res = bootstrap(
    (all_final_returns,), 
    np.median, 
    n_resamples=1000, 
    random_state=rng,
    confidence_level=0.95
)

median_val = np.median(all_final_returns)
ci_low, ci_high = res.confidence_interval

print(f"전체 중앙값: {median_val:.1f} (= {median_val/100:.2f}배)")
print(f"95% 신뢰구간: [{ci_low:.1f}, {ci_high:.1f}]")
print(f"           = [{ci_low/100:.2f}배, {ci_high/100:.2f}배]")
print()

# 클러스터별 Bootstrap
print("클러스터별 중앙값 95% 신뢰구간:")
for c in range(kmeans.n_clusters):
    cluster_tickers = cluster_df[cluster_df['Cluster'] == c]['Ticker']
    final_vals = normalized_1[cluster_tickers].iloc[-1].values
    
    if len(final_vals) >= 10:  # 최소 샘플 수 확인
        res_c = bootstrap(
            (final_vals,), 
            np.median, 
            n_resamples=1000, 
            random_state=rng,
            confidence_level=0.95
        )
        med = np.median(final_vals)
        ci_l, ci_h = res_c.confidence_interval
        print(f"  Cluster {c}: {med:.0f} [{ci_l:.0f}, {ci_h:.0f}] = {med/100:.1f}x [{ci_l/100:.1f}x, {ci_h/100:.1f}x]")
    else:
        print(f"  Cluster {c}: 샘플 수 부족 (n={len(final_vals)})")


In [ ]:
# 섹터 2 클러스터 요약
for c in range(kmeans.n_clusters):
    cluster_tickers = cluster_df_2[cluster_df_2['Cluster'] == c]['Ticker']
    cluster_data = normalized_2[cluster_tickers]
    
    # 최종 수익률
    final_vals = cluster_data.iloc[-1]
    
    print(f"=== Cluster {c} ({len(cluster_tickers)} stocks) ===")
    print(f"  중앙값 최종: {cluster_data.median(axis=1).iloc[-1]:.1f} (={cluster_data.median(axis=1).iloc[-1]/100:.1f}x)")
    print(f"  평균 최종:   {final_vals.mean():.1f} (={final_vals.mean()/100:.1f}x)")
    print(f"  범위:        {final_vals.min():.1f} ~ {final_vals.max():.1f}")
    
    # 섹터 구성
    cluster_sectors = sector_df_2[cluster_tickers].value_counts()
    print(f"  섹터 구성:")
    for sector, count in cluster_sectors.items():
        print(f"    {sector}: {count}개")
    
    # 대표 종목 (최종 정규화 값 기준 상위 3)
    top3 = final_vals.nlargest(3)
    print(f"  상위 3종목: {', '.join([f'{t}({v:.0f})' for t, v in top3.items()])}")
    print()


In [ ]:
# === 그룹 2 통계적 검증 ===
print("=" * 50)
print("그룹 2 통계적 검증: 클러스터 간 차이가 유의미한가?")
print("=" * 50)

# 1. Kruskal-Wallis 검정
print("\n[1] Kruskal-Wallis 검정 (클러스터 간 수익률 차이)")
print("-" * 40)

cluster_returns_2 = []
for c in range(kmeans_2.n_clusters):
    cluster_tickers = cluster_df_2[cluster_df_2['Cluster'] == c]['Ticker']
    final_vals = normalized_2[cluster_tickers].iloc[-1]
    cluster_returns_2.append(final_vals.values)

stat_2, p_value_2 = kruskal(*cluster_returns_2)
print(f"H-statistic: {stat_2:.2f}")
print(f"p-value: {p_value_2:.2e}")
if p_value_2 < 0.05:
    print("→ 결론: 클러스터 간 수익률 차이가 통계적으로 유의미함 (p < 0.05)")
else:
    print("→ 결론: 클러스터 간 수익률 차이가 유의미하지 않음")

# 2. 카이제곱 검정
print("\n[2] 카이제곱 검정 (섹터-클러스터 연관성)")
print("-" * 40)

cluster_sector_df_2 = pd.DataFrame({
    'Cluster': cluster_df_2.set_index('Ticker')['Cluster'],
    'Sector': sector_df_2
}).dropna()

contingency_table_2 = pd.crosstab(cluster_sector_df_2['Sector'], cluster_sector_df_2['Cluster'])
print("섹터 × 클러스터 교차표:")
print(contingency_table_2)
print()

chi2_2, p_value_chi_2, dof_2, expected_2 = chi2_contingency(contingency_table_2)
print(f"Chi-square: {chi2_2:.2f}")
print(f"자유도: {dof_2}")
print(f"p-value: {p_value_chi_2:.2e}")
if p_value_chi_2 < 0.05:
    print("→ 결론: 섹터와 클러스터 간 연관성이 통계적으로 유의미함 (p < 0.05)")
else:
    print("→ 결론: 섹터와 클러스터 간 연관성이 유의미하지 않음 (독립적)")

# 3. Bootstrap 신뢰구간
print("\n[3] Bootstrap 신뢰구간 (중앙값 수익률)")
print("-" * 40)

all_final_returns_2 = normalized_2.iloc[-1].values
res_2 = bootstrap(
    (all_final_returns_2,), 
    np.median, 
    n_resamples=1000, 
    random_state=rng,
    confidence_level=0.95
)

median_val_2 = np.median(all_final_returns_2)
ci_low_2, ci_high_2 = res_2.confidence_interval

print(f"전체 중앙값: {median_val_2:.1f} (= {median_val_2/100:.2f}배)")
print(f"95% 신뢰구간: [{ci_low_2:.1f}, {ci_high_2:.1f}]")
print(f"           = [{ci_low_2/100:.2f}배, {ci_high_2/100:.2f}배]")


## 6. 상대강도 분석 — 시장을 이긴 진짜 승자 찾기

종목 가격 / S&P 500 지수 가격으로 상대강도를 계산한다. 이 수치가 우상향하는 구간이 **시장보다 강한 진짜 랠리**이며, 단순히 시장과 함께 오른 종목과 초과수익을 낸 종목을 구분할 수 있다.

In [ ]:
# 그룹 1
# RSI 계산하기: 상승 시 주가 상승폭의 평균 / 하락 시 주가 하락폭의 평균
delta = close_df_1.diff()
gains = delta.where(delta > 0, 0)
losses = -delta.where(delta < 0, 0)

# 14일 기준으로 이동평균 계산
n = 14
avg_gain = gains.rolling(window=n).mean()
avg_loss = losses.rolling(window=n).mean()

rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))

rsi.head(20)

In [ ]:
# 그룹 2
# RSI 계산하기: 상승 시 주가 상승폭의 평균 / 하락 시 주가 하락폭의 평균
delta_2 = close_df_2.diff()
gains_2 = delta_2.where(delta_2 > 0, 0)
losses_2 = -delta_2.where(delta_2 < 0, 0)

# 14일 기준으로 이동평균 계산
n = 14
avg_gain_2 = gains_2.rolling(window=n).mean()
avg_loss_2 = losses_2.rolling(window=n).mean()

rs_2 = avg_gain_2 / avg_loss_2
rsi_2 = 100 - (100 / (1 + rs_2))
rsi_2.head(20)

In [ ]:
# 그룹 1
# 상대강도(RS) = 종목 정규화 가격 / NASDAQ 정규화 가격
# 시작점=1로 정규화하여 비교
# 1보다 위 = 나스닥 초과수익
nasdaq = close_df_1['^IXIC'] if '^IXIC' in close_df_1.columns else None

if nasdaq is not None:
    nasdaq_norm = nasdaq / nasdaq.iloc[0]
    rs_df = normalized_1.div(nasdaq_norm * 100, axis=0)  # normalized_1은 100 기준이므로 맞춤
else:
    print("NASDAQ 데이터가 없습니다")
    rs_df = None

if rs_df is not None:
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.set_yscale('log')
    
    # 섹터별로 개별 종목 RS 그리기
    for sector, tickers in sector_df.groupby(sector_df):
        color = sector_colors.get(sector, 'grey')
        for ticker in tickers.index:
            if ticker in rs_df.columns and ticker != '^IXIC':
                ax.plot(rs_df[ticker], color=color, alpha=0.25, linewidth=0.6)
        # 범례용 더미 라인
        count = len([t for t in tickers.index if t in rs_df.columns and t != '^IXIC'])
        if count > 0:
            ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({count})')
    
    # 전체 RS 중앙값
    rs_median = rs_df.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    ax.plot(rs_median, color='black', linewidth=2.5, linestyle='--', label='Median RS')
    
    # 기준선: RS=1은 나스닥과 동일 수익률
    ax.axhline(1, color='red', linestyle='-', linewidth=2, label='RS=1 (NASDAQ)')
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Relative Strength vs NASDAQ (Log)')
    ax.set_title('Relative Strength by Sector')
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    plt.tight_layout()
    plt.show()

In [ ]:
# 그룹 2
# 상대강도(RS) = 종목 정규화 가격 / NASDAQ 정규화 가격
# 시작점=1로 정규화하여 비교
# 1보다 위 = 나스닥 초과수익
nasdaq = close_df_2['^IXIC'] if '^IXIC' in close_df_2.columns else None

if nasdaq is not None:
    nasdaq_norm = nasdaq / nasdaq.iloc[0]
    rs_df_2 = normalized_2.div(nasdaq_norm * 100, axis=0)  # normalized_2은 100 기준이므로 맞춤
else:
    print("NASDAQ 데이터가 없습니다")
    rs_df_2 = None

if rs_df_2 is not None:
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.set_yscale('log')
    
    # 섹터별로 개별 종목 RS 그리기
    for sector, tickers in sector_df_2.groupby(sector_df_2):
        color = sector_colors.get(sector, 'grey')
        for ticker in tickers.index:
            if ticker in rs_df_2.columns and ticker != '^IXIC':
                ax.plot(rs_df_2[ticker], color=color, alpha=0.25, linewidth=0.6)
        # 범례용 더미 라인
        count = len([t for t in tickers.index if t in rs_df_2.columns and t != '^IXIC'])
        if count > 0:
            ax.plot([], [], color=color, linewidth=2, label=f'{sector} ({count})')
    
    # 전체 RS 중앙값
    rs_median_2 = rs_df_2.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    ax.plot(rs_median_2, color='black', linewidth=2.5, linestyle='--', label='Median RS')
    
    # 기준선: RS=1은 나스닥과 동일 수익률
    ax.axhline(1, color='red', linestyle='-', linewidth=2, label='RS=1 (NASDAQ)')

    
    ax.set_xlabel('Date')
    ax.set_ylabel('Relative Strength vs NASDAQ (Log)')
    ax.set_title('Relative Strength by Sector')
    ax.legend(fontsize=7, loc='upper left', ncol=2)
    plt.tight_layout()
    plt.show()

In [ ]:
# 그룹 1
# 기간별 RS 차트 (4개 구간으로 분할)
if rs_df is not None:
    # 기간 정의
    periods = [
        ('1988.10 ~ 1991.12', '1988-10-01', '1991-12-31'),
        ('1992.01 ~ 1994.12', '1992-01-01', '1994-12-31'),
        ('1995.01 ~ 1997.12', '1995-01-01', '1997-12-31'),
        ('1998.01 ~ 2000.03', '1998-01-01', '2000-03-31'),
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, (period_name, start, end) in enumerate(periods):
        ax = axes[idx]
        ax.set_yscale('log')
        
        # 해당 기간 데이터 슬라이싱
        rs_period = rs_df.loc[start:end]
        
        # 기간 시작점 기준으로 다시 정규화 (각 기간의 시작=1)
        rs_period_norm = rs_period / rs_period.iloc[0]
        
        # 섹터별로 개별 종목 RS 그리기
        for sector, tickers in sector_df.groupby(sector_df):
            color = sector_colors.get(sector, 'grey')
            for ticker in tickers.index:
                if ticker in rs_period_norm.columns and ticker != '^IXIC':
                    ax.plot(rs_period_norm[ticker], color=color, alpha=0.5, linewidth=0.6)
        
        # 해당 기간 RS 중앙값
        rs_median = rs_period_norm.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
        ax.plot(rs_median, color='black', linewidth=2.5, linestyle='--', label='Median RS')
        
        # 기준선
        ax.axhline(1, color='red', linestyle='-', linewidth=1.5, label='RS=1 (NASDAQ)')
        
        ax.set_title(f'{period_name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('RS (period start=1)')
        ax.legend(fontsize=8, loc='upper left')
    
    plt.suptitle('Relative Strength by Period', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# 그룹 2
# 기간별 RS 차트 (4개 구간으로 분할)
if rs_df_2 is not None:
    # 기간 정의
    periods = [
        ('1999.03 ~ 1999.05', '1999-03-01', '1999-05-31'),
        ('1999.06 ~ 1999.08', '1999-06-01', '1999-08-31'),
        ('1999.09 ~ 1999.11', '1999-09-01', '1999-11-30'),
        ('1999.12 ~ 2000.03', '1999-12-01', '2000-03-31'),
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, (period_name, start, end) in enumerate(periods):
        ax = axes[idx]
        ax.set_yscale('log')
        
        # 해당 기간 데이터 슬라이싱
        rs_period_2 = rs_df_2.loc[start:end]
        
        # 기간 시작점 기준으로 다시 정규화 (각 기간의 시작=1)
        rs_period_norm = rs_period_2 / rs_period_2.iloc[0]
        
        # 섹터별로 개별 종목 RS 그리기
        for sector, tickers in sector_df_2.groupby(sector_df_2):
            color = sector_colors.get(sector, 'grey')
            for ticker in tickers.index:
                if ticker in rs_period_norm.columns and ticker != '^IXIC':
                    ax.plot(rs_period_norm[ticker], color=color, alpha=0.5, linewidth=0.6)
        
        # 해당 기간 RS 중앙값
        rs_median = rs_period_norm.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
        ax.plot(rs_median, color='black', linewidth=2.5, linestyle='--', label='Median RS')
        
        # 기준선
        ax.axhline(1, color='red', linestyle='-', linewidth=1.5, label='RS=1 (NASDAQ)')
        
        ax.set_title(f'{period_name}', fontsize=12, fontweight='bold')
        ax.set_ylabel('RS (period start=1)')
        ax.legend(fontsize=8, loc='upper left')
    
    plt.suptitle('Relative Strength by Period', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
def classify_rs_pattern(rs_series, median_series, threshold=0.1):
    """
    RS 패턴을 4가지로 분류
    1: 항상 위 (전 기간 중앙값/1 위)
    2: 골든크로스 (약하다가 치고 오름)
    3: 데드크로스 (강하다가 치고 내려감)
    4: 항상 아래 (전 기간 중앙값/1 아래)
    """
    rs = rs_series.dropna()
    med = median_series.loc[rs.index]
    
    if len(rs) < 10:
        return None
    
    # 기간을 전반/후반으로 나눔
    mid = len(rs) // 2
    first_half = rs.iloc[:mid]
    second_half = rs.iloc[mid:]
    med_first = med.iloc[:mid]
    med_second = med.iloc[mid:]
    
    # 전반/후반 평균이 중앙값 대비 위인지 아래인지
    first_above = (first_half > med_first).mean() > 0.6
    first_below = (first_half < med_first).mean() > 0.6
    second_above = (second_half > med_second).mean() > 0.6
    second_below = (second_half < med_second).mean() > 0.6
    
    if first_above and second_above:
        return 1  # 항상 위
    elif first_below and second_above:
        return 2  # 골든크로스
    elif first_above and second_below:
        return 3  # 데드크로스
    elif first_below and second_below:
        return 4  # 항상 아래
    else:
        return 0  # 분류 불가 (혼재)

# 그룹 1
# 기간별로 분류 실행
periods = [
    ('1988.10~1991.12', '1988-10-01', '1991-12-31'),
    ('1992.01~1994.12', '1992-01-01', '1994-12-31'),
    ('1995.01~1997.12', '1995-01-01', '1997-12-31'),
    ('1998.01~2000.03', '1998-01-01', '2000-03-31'),
]

pattern_results = {}

for period_name, start, end in periods:
    rs_period = rs_df.loc[start:end]
    rs_period_norm = rs_period / rs_period.iloc[0]
    rs_median = rs_period_norm.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    
    period_patterns = {}
    for ticker in rs_period_norm.columns:
        if ticker == '^IXIC':
            continue
        pattern = classify_rs_pattern(rs_period_norm[ticker], rs_median)
        if pattern is not None:
            period_patterns[ticker] = pattern
    
    pattern_results[period_name] = period_patterns
    
    # 결과 요약
    from collections import Counter
    counts = Counter(period_patterns.values())
    print(f"\n=== {period_name} ===")
    print(f"1. 항상 위: {counts.get(1, 0)}개")
    print(f"2. 골든크로스: {counts.get(2, 0)}개")
    print(f"3. 데드크로스: {counts.get(3, 0)}개")
    print(f"4. 항상 아래: {counts.get(4, 0)}개")
    print(f"0. 분류불가: {counts.get(0, 0)}개")

In [ ]:
# 그룹 2
# 기간별로 분류 실행
periods = [
    ('1999.03 ~ 1999.05', '1999-03-01', '1999-05-31'),
    ('1999.06 ~ 1999.08', '1999-06-01', '1999-08-31'),
    ('1999.09 ~ 1999.11', '1999-09-01', '1999-11-30'),
    ('1999.12 ~ 2000.03', '1999-12-01', '2000-03-31'),
]

pattern_results_2 = {}

for period_name, start, end in periods:
    rs_period_2 = rs_df_2.loc[start:end]
    rs_period_norm_2 = rs_period_2 / rs_period_2.iloc[0]
    rs_median_2 = rs_period_norm_2.drop(columns=['^IXIC'], errors='ignore').median(axis=1)
    
    period_patterns_2 = {}
    for ticker in rs_period_norm_2.columns:
        if ticker == '^IXIC':
            continue
        pattern = classify_rs_pattern(rs_period_norm_2[ticker], rs_median_2)
        if pattern is not None:
            period_patterns_2[ticker] = pattern
    
    pattern_results_2[period_name] = period_patterns_2
    
    # 결과 요약
    from collections import Counter
    counts = Counter(period_patterns_2.values())
    print(f"\n=== {period_name} ===")
    print(f"1. 항상 위: {counts.get(1, 0)}개")
    print(f"2. 골든크로스: {counts.get(2, 0)}개")
    print(f"3. 데드크로스: {counts.get(3, 0)}개")
    print(f"4. 항상 아래: {counts.get(4, 0)}개")
    print(f"0. 분류불가: {counts.get(0, 0)}개")

In [ ]:
# 그룹 1
# 각 기간별 그룹의 섹터 구성 분석
for period_name, patterns in pattern_results.items():
    print(f"\n{'='*60}")
    print(f"=== {period_name} ===")
    print(f"{'='*60}")
    
    for group_num, group_name in [(1, "항상 위"), (2, "골든크로스"), (3, "데드크로스"), (4, "항상 아래")]:
        tickers = [t for t, p in patterns.items() if p == group_num]
        if not tickers:
            continue
        
        print(f"\n--- {group_num}. {group_name} ({len(tickers)}개) ---")
        
        # 섹터 분포
        sectors = sector_df[tickers].value_counts()
        print("섹터 분포:")
        for sector, count in sectors.head(5).items():
            print(f"  {sector}: {count}개")
        
        # 대표 종목 (RS 최종값 기준)
        rs_period = rs_df.loc[periods[[p[0] for p in periods].index(period_name)][1]:periods[[p[0] for p in periods].index(period_name)][2]]
        if len(tickers) > 0 and len(rs_period) > 0:
            final_rs = rs_period[tickers].iloc[-1].sort_values(ascending=False)
            print(f"대표 종목 (RS 상위): {list(final_rs.head(3).index)}")

In [ ]:
# 그룹 2
# 각 기간별 그룹의 섹터 구성 분석
for period_name, patterns in pattern_results_2.items():
    print(f"\n{'='*60}")
    print(f"=== {period_name} ===")
    print(f"{'='*60}")
    
    for group_num, group_name in [(1, "항상 위"), (2, "골든크로스"), (3, "데드크로스"), (4, "항상 아래")]:
        tickers = [t for t, p in patterns.items() if p == group_num]
        if not tickers:
            continue
        
        print(f"\n--- {group_num}. {group_name} ({len(tickers)}개) ---")
        
        # 섹터 분포
        sectors_2 = sector_df_2[tickers].value_counts()
        print("섹터 분포:")
        for sector, count in sectors_2.head(5).items():
            print(f"  {sector}: {count}개")
        
        # 대표 종목 (RS 최종값 기준)
        rs_period_2 = rs_df_2.loc[periods[[p[0] for p in periods].index(period_name)][1]:periods[[p[0] for p in periods].index(period_name)][2]]
        if len(tickers) > 0 and len(rs_period_2) > 0:
            final_rs_2 = rs_period_2[tickers].iloc[-1].sort_values(ascending=False)
            print(f"대표 종목 (RS 상위): {list(final_rs_2.head(3).index)}")

In [ ]:
# 그룹 1
# 전 기간 걸쳐 일관된 패턴을 보이는 종목 찾기
from collections import defaultdict

# 종목별 기간별 패턴 취합
ticker_patterns = defaultdict(list)
for period_name, patterns in pattern_results.items():
    for ticker, pattern in patterns.items():
        ticker_patterns[ticker].append((period_name, pattern))

# 항상 위 (4개 기간 모두 1번)
always_above = [t for t, patterns in ticker_patterns.items() 
                if len(patterns) == 4 and all(p[1] == 1 for p in patterns)]

# 항상 아래 (4개 기간 모두 4번)
always_below = [t for t, patterns in ticker_patterns.items() 
                if len(patterns) == 4 and all(p[1] == 4 for p in patterns)]

# 상승 추세 (초기에 아래→후기에 위)
rising = [t for t, patterns in ticker_patterns.items()
          if len(patterns) == 4 and patterns[0][1] in [4, 0] and patterns[3][1] == 1]

# 하락 추세 (초기에 위→후기에 아래)
falling = [t for t, patterns in ticker_patterns.items()
           if len(patterns) == 4 and patterns[0][1] == 1 and patterns[3][1] in [4, 0]]

print("=== 전 기간 일관된 패턴 ===\n")

print(f"1. 12년간 항상 나스닥을 이긴 종목 ({len(always_above)}개):")
for t in always_above:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n2. 12년간 항상 나스닥에 진 종목 ({len(always_below)}개):")
for t in always_below:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n3. 초기 약세 → 후기 강세로 전환 ({len(rising)}개):")
for t in rising:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n4. 초기 강세 → 후기 약세로 전환 ({len(falling)}개):")
for t in falling:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

In [ ]:
# 그룹 2
# 전 기간 걸쳐 일관된 패턴을 보이는 종목 찾기

# 종목별 기간별 패턴 취합
ticker_patterns_2 = defaultdict(list)
for period_name, patterns in pattern_results_2.items():
    for ticker, pattern in patterns.items():
        ticker_patterns_2[ticker].append((period_name, pattern))

# 항상 위 (4개 기간 모두 1번)
always_above = [t for t, patterns in ticker_patterns_2.items() 
                if len(patterns) == 4 and all(p[1] == 1 for p in patterns)]

# 항상 아래 (4개 기간 모두 4번)
always_below = [t for t, patterns in ticker_patterns_2.items() 
                if len(patterns) == 4 and all(p[1] == 4 for p in patterns)]

# 상승 추세 (초기에 아래→후기에 위)
rising = [t for t, patterns in ticker_patterns_2.items()
          if len(patterns) == 4 and patterns[0][1] in [4, 0] and patterns[3][1] == 1]

# 하락 추세 (초기에 위→후기에 아래)
falling = [t for t, patterns in ticker_patterns_2.items()
           if len(patterns) == 4 and patterns[0][1] == 1 and patterns[3][1] in [4, 0]]

print("=== 전 기간 일관된 패턴 ===\n")

print(f"1. 12년간 항상 나스닥을 이긴 종목 ({len(always_above)}개):")
for t in always_above:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n2. 12년간 항상 나스닥에 진 종목 ({len(always_below)}개):")
for t in always_below:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n3. 초기 약세 → 후기 강세로 전환 ({len(rising)}개):")
for t in rising:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

print(f"\n4. 초기 강세 → 후기 약세로 전환 ({len(falling)}개):")
for t in falling:
    sector = sector_df.get(t, 'Unknown')
    print(f"  {t} ({sector})")

### RS 분석 결과

**12년간 일관되게 나스닥을 이긴 종목**: AB, AEG (Financial Services), LEG (Consumer Cyclical) — 단 3개

**초기 약세 → 후기 강세 전환 (25개)**: AAPL, AMAT, BBY, COST, SPGI 등

**초기 강세 → 후기 약세 전환 (24개)**: BA, NSC, BEN, SLB 등 전통 산업재/에너지

---

### 왜 이런 결과가 나왔는가

1. **기술 테마의 진짜 수혜자는 금융 인프라였다** — 버블 당시 IPO, M&A 자문, 자산 관리 수요가 폭발하면서 AB, SPGI 같은 종목이 테크보다 안정적으로 나스닥을 이김

2. **혁신주의 언더퍼폼은 실패가 아니라 캐즘이다** — AAPL, BBY, AMAT는 초기 7~8년간 지수에 졌지만, 혁신이 실제 수익으로 증명되는 순간 폭발함. 지수 대비 언더퍼폼 구간이 곧 매수 기회일 수 있음

3. **Late Bloomer는 기술로 비용구조를 혁신한 전통 섹터에서 나온다** — BBY(IT기기 유통 수요), COST(공급망 효율화)처럼 기술 자체가 아니라 기술을 활용한 기업

4. **자본의 선호가 물리적 자산 → 무형 자산으로 이동했다** — BA, NSC 같은 설비 집약적 산업재는 섹터가 좋아도 RS 회복이 어려웠음. 디지털 시대에 물리적 자산에 묶인 기업의 한계

## 7. 결론

### 그룹 1 vs 그룹 2 비교

| 지표 | 그룹 1 (12년 장기) | 그룹 2 (버블 말기 1년) |
|------|-------------------|----------------------|
| 중앙값 수익률 | 3.88x (288%) | 1.08x (8%) |
| 95% 신뢰구간 | **[3.08x, 4.35x]** | **[1.02x, 1.12x]** |
| 손실 종목 비율 | 2% (4개) | 37% (125개) |
| 성장 클러스터 1위 섹터 | Financial Services | Technology |

### 통계적 검증 결과

| 검증 | 그룹 1 | 그룹 2 | 해석 |
|------|--------|--------|------|
| Kruskal-Wallis | **유의미** (p < 0.05) | **유의미** (p < 0.05) | 클러스터 구분이 통계적으로 의미있음 |
| 카이제곱 (섹터-클러스터) | **독립** (p > 0.05) | **연관** (p < 0.05) | 그룹1은 섹터 무관, 그룹2는 Tech 집중 |
| Bootstrap 95% CI | [3.08x, 4.35x] | [1.02x, 1.12x] | **두 그룹 CI가 겹치지 않음 → 차이 유의미** |

### 핵심 발견

**장기 랠리 (그룹 1):**
1. **섹터와 성과는 독립적** — 카이제곱 검정 결과, 특정 섹터가 특정 클러스터에 집중되지 않음. 섹터보다 종목 선별이 중요
2. **최악의 경우에도 3배** — Bootstrap 하한이 3.08x. Survivorship Bias를 감안해도 장기 보유 전략의 유효성 확인
3. **소수의 승자가 시장을 견인** — 중앙값 3.9배 vs 평균 7.3배

**버블 말기 (그룹 2):**
1. **섹터와 성과가 연관됨** — 카이제곱 검정 결과 유의미. Tech/바이오가 성장 클러스터에 집중
2. **본전치기가 최선** — 중앙값 1.08x, 신뢰구간도 [1.02x, 1.12x]로 좁음. 대부분이 비슷하게 끝남
3. **양극화 극심** — 37%가 손실인데 일부는 4배. "운 좋으면 4배, 아니면 -50%" 구조

### 한계

- **Survivorship Bias**: 상장폐지 종목 누락으로 손실 비율이 과소추정됨
- 그룹 1의 "2% 손실"은 실제보다 낙관적일 가능성 높음
- 추후 FMP API로 상장폐지 종목 포함 재분석 필요

### 후속 과제

- [ ] FMP API로 데이터 소스 교체 → Survivorship Bias 해소
- [x] ~~그룹 2 분석~~ → 완료
- [x] ~~통계적 검증~~ → Kruskal-Wallis, 카이제곱, Bootstrap 완료
- [ ] 2009~2021 빅테크 랠리 분석 → 패턴 재현 검증